In [3]:
import csv
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver

def get_url(search_term):
    '''Generate a url from search term'''
    template = "https://www.amazon.in/s?k={}&ref=nb_sb_noss_2"
    search_term = search_term.replace(' ', '+')
    
    # add term query to url
    url = template.format(search_term)
    
    return url

def extract_record(item):
    '''Extract and return data from a single record'''
    
    atag = item.h2.a
    description = atag.text.strip()
    
    try:
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
    
    try:
        rating = item.i.text
    except AttributeError:
        rating = ''
    
    result = (description, price, rating)
    
    return result

def main(search_term):
    '''Run main program routine'''
    # startup the web driver
    driver = webdriver.Chrome('/home/soham/pypypy/chromedriver')
    
    records = []
    url = get_url(search_term)

    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    results = soup.find_all('div', {'data-component-type': 's-search-result'})

    for item in results:
        record = extract_record(item)
        if record:
            records.append(record)
    driver.close()

    #save the data

    with open('results.csv', 'w', newline = '', encoding = 'utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description', 'Price', 'Rating'])
        writer.writerows(records)
        
    df = pd.read_csv('results.csv')
    print (df) 
        
n = input("Enter search query: ")
main(n)

Enter search query: macbook
                                          Description      Price  \
0   2020 Apple MacBook Pro (13.3-inch/33.78 cm, Ap...  ₹1,12,900   
1   2019 Apple MacBook Pro (16-inch/40.65 cm, 16GB...  ₹2,15,990   
2   2020 Apple MacBook Pro (13.3-inch/33.78 cm, Ap...  ₹1,12,900   
3   2019 Apple MacBook Pro (16-inch/40.65 cm, 16GB...  ₹1,79,990   
4   2019 Apple MacBook Pro (16-inch/40.65 cm, 16GB...  ₹2,15,990   
5   ASUS ZenBook Duo UX481 Intel Core i5 10th Gen ...    ₹84,990   
6   ASUS ZenBook Pro Duo Intel Core i7-10750H 10th...  ₹2,24,494   
7   Microsoft Surface Laptop 3 AMD Ryzen 5 15-inch...    ₹97,503   
8   Microsoft Surface Laptop Go 10th Gen Intel Cor...    ₹69,490   
9   HP Pavilion 13 11th Gen Intel Core i5 13.3" (3...    ₹69,990   
10  Microsoft Surface Pro 6 LGP-00015 12.3 inch To...    ₹82,999   
11  Microsoft Surface GO 2 STQ-00013 10.1" (26.54 ...    ₹57,490   
12  Lenovo Yoga Slim 7 Carbon 11th Gen Intel Core ...  ₹1,17,493   
13  VAIO SE14 NP14V3